In [1]:
import pandas as pd
import re
samples_raw=pd.read_json('./dataset/train.jsonl',lines=True)
samples_raw.answer.apply(lambda x: re.sub("(<<).*?(>>)", "", x).split('####')[0].rstrip())


0       Natalia sold 48/2 = 24 clips in May.\nNatalia ...
1       Weng earns 12/60 = $0.2 per minute.\nWorking 5...
2       In the beginning, Betty has only 100 / 2 = $50...
3       Maila read 12 x 2 = 24 pages today.\nSo she wa...
4       He writes each friend 3*2=6 pages a week\nSo h...
                              ...                        
7468    For the distance she traveled, Elise paid 23 -...
7469    He makes $.5 profit on each bracelet because 1...
7470    Tony can skip at twice the speed that Bruce ca...
7471    Janet needs 35 lunches for the kids + 5 for th...
7472    If Anika is 30 now, in 15 years, she'll be 30+...
Name: answer, Length: 7473, dtype: object

In [50]:
samples_raw.head()

,question,answer
0,Natalia sold clips to 48 of her friends in Apr...,Natalia sold 48/2 = <<48/2=24>>24 clips in May...
1,Weng earns $12 an hour for babysitting. Yester...,Weng earns 12/60 = $<<12/60=0.2>>0.2 per minut...
2,Betty is saving money for a new wallet which c...,"In the beginning, Betty has only 100 / 2 = $<<..."
3,"Julie is reading a 120-page book. Yesterday, s...",Maila read 12 x 2 = <<12*2=24>>24 pages today....
4,James writes a 3-page letter to 2 different fr...,He writes each friend 3*2=<<3*2=6>>6 pages a w...


In [45]:
# samples_raw.insert(samples_raw.shape[1], 'd', "")
for i in range(len(samples_raw)):
    # print(i)
    samples_raw.loc[i]['d']=['aa']
    # break

/tmp/ipykernel_1737238/3018189026.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  samples_raw.loc[i]['d']=['aa']


In [41]:
samples_raw.head()

,question,answer,d
0,Natalia sold clips to 48 of her friends in Apr...,Natalia sold 48/2 = 24 clips in May.\nNatalia ...,bb
1,Weng earns $12 an hour for babysitting. Yester...,Weng earns 12/60 = $0.2 per minute.\nWorking 5...,bb
2,Betty is saving money for a new wallet which c...,"In the beginning, Betty has only 100 / 2 = $50...",bb
3,"Julie is reading a 120-page book. Yesterday, s...",Maila read 12 x 2 = 24 pages today.\nSo she wa...,bb
4,James writes a 3-page letter to 2 different fr...,He writes each friend 3*2=6 pages a week\nSo h...,bb


In [42]:
samples_raw['ss']=samples_raw['d']

In [43]:
samples_raw['d']=[]

In [46]:
samples_raw.head()

,question,answer,d,ss
0,Natalia sold clips to 48 of her friends in Apr...,Natalia sold 48/2 = 24 clips in May.\nNatalia ...,1,bb
1,Weng earns $12 an hour for babysitting. Yester...,Weng earns 12/60 = $0.2 per minute.\nWorking 5...,1,bb
2,Betty is saving money for a new wallet which c...,"In the beginning, Betty has only 100 / 2 = $50...",1,bb
3,"Julie is reading a 120-page book. Yesterday, s...",Maila read 12 x 2 = 24 pages today.\nSo she wa...,1,bb
4,James writes a 3-page letter to 2 different fr...,He writes each friend 3*2=6 pages a week\nSo h...,1,bb


In [2]:
from datasets import load_dataset
data_files = {'train': './dataset/train.jsonl', 'test': './dataset/test.jsonl'}
dataset = load_dataset('json', data_files=data_files, split='train').select(range(25))
dataset = dataset.rename_column('question', 'input')
dataset = dataset.rename_column('answer', 'label')
dataset = dataset.map(lambda x: {'input': x['input'],
                                     'label': re.sub("(<<).*?(>>)", "", x['label']).split('####')[0].rstrip().replace('\n','')})
print(dataset)


Found cached dataset json (/home/chenlida/.cache/huggingface/datasets/json/default-c9569cfc01ae9d34/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
Loading cached processed dataset at /home/chenlida/.cache/huggingface/datasets/json/default-c9569cfc01ae9d34/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-1ddb9b679f6a4107.arrow


Dataset({
    features: ['input', 'label'],
    num_rows: 25
})


In [3]:
PROMPT_TEMPLATE = \
"""Your task is to add calls to a Calculator API to a piece of text.
The calls should help you get information required to complete the text. You can call the API by writing "[Calculator(expression)]" where "expression" is the expression to be computed. Here are some examples of API calls: 
Input: From this, we have 4 * 30 minutes = 120 minutes.
Output: From this, we have 4 * 30 minutes = [Calculator(4 * 30)] 120 minutes.
Input: {}
Output:"""

In [4]:

def prepare_dataset_for_sampling(dataset, tokenizer):
    prompts_dataset = dataset.map(lambda x: {'prompt': PROMPT_TEMPLATE.format(x['label']) + x['label'],
                                             'label': x['label']})
    encoded_dataset = prompts_dataset.map(lambda x: tokenizer(x['prompt'],truncation=True, padding=False))
    encoded_dataset = encoded_dataset.map(lambda x: {'target_mask': (len(x['input_ids']) - len(tokenizer(x['label'])['input_ids'])) * [0] + len(tokenizer(x['label'])['input_ids']) * [1]})
    
    encoded_dataset.set_format(columns=['input_ids', 'attention_mask', 'target_mask'], type='torch')
    
    print(tokenizer.decode(encoded_dataset[0]['input_ids']))
    return encoded_dataset
from transformers import GPT2Tokenizer,AutoTokenizer
# tokenizer=GPT2Tokenizer.from_pretrained("EleutherAI/gpt-j-6B")
tokenizer = AutoTokenizer.from_pretrained("hf-internal-testing/tiny-random-gptj")
print(len(tokenizer))
tokenizer.pad_token=tokenizer.eos_token
dataset_n=prepare_dataset_for_sampling(dataset,tokenizer)

2023-03-14 12:03:52.354353: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-14 12:03:52.512852: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-14 12:03:52.542210: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-14 12:03:53.086393: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

1000
Your task is to add calls to a Calculator API to a piece of text.
The calls should help you get information required to complete the text. You can call the API by writing "[Calculator(expression)]" where "expression" is the expression to be computed. Here are some examples of API calls: 
Input: From this, we have 4 * 30 minutes = 120 minutes.
Output: From this, we have 4 * 30 minutes = [Calculator(4 * 30)] 120 minutes.
Input: Natalia sold 48/2 = 24 clips in May.Natalia sold 48+24 = 72 clips altogether in April and May.
Output:Natalia sold 48/2 = 24 clips in May.Natalia sold 48+24 = 72 clips altogether in April and May.


In [5]:
import torch
from transformers import AutoTokenizer, GPTJForCausalLM

tokenizer = AutoTokenizer.from_pretrained("hf-internal-testing/tiny-random-gptj")
model = GPTJForCausalLM.from_pretrained("hf-internal-testing/tiny-random-gptj")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
print(inputs)
outputs = model(**inputs, labels=inputs["input_ids"])
loss = outputs.loss
logits = outputs.logits
tk=logits.topk(2)
print(tk)
idx,value=tk.indices,tk.values

Some weights of the model checkpoint at hf-internal-testing/tiny-random-gptj were not used when initializing GPTJForCausalLM: ['h.1.mlp.fc_in.bias', 'h.2.attn.q_proj.weight', 'h.4.mlp.fc_in.weight', 'h.4.mlp.fc_out.bias', 'h.0.ln_1.bias', 'wte.weight', 'h.4.attn.v_proj.weight', 'h.2.attn.bias', 'h.4.attn.k_proj.weight', 'h.1.mlp.fc_in.weight', 'h.0.ln_1.weight', 'h.0.mlp.fc_in.weight', 'h.2.attn.masked_bias', 'h.0.mlp.fc_in.bias', 'h.2.ln_1.weight', 'score.weight', 'h.2.mlp.fc_in.bias', 'h.4.attn.q_proj.weight', 'h.2.mlp.fc_out.weight', 'h.1.attn.v_proj.weight', 'h.2.attn.k_proj.weight', 'h.0.attn.k_proj.weight', 'h.1.ln_1.weight', 'ln_f.weight', 'h.2.mlp.fc_out.bias', 'h.3.ln_1.bias', 'h.3.mlp.fc_in.bias', 'h.1.mlp.fc_out.weight', 'h.3.attn.k_proj.weight', 'h.1.attn.k_proj.weight', 'h.4.attn.masked_bias', 'h.3.attn.out_proj.weight', 'ln_f.bias', 'h.3.mlp.fc_out.weight', 'h.4.attn.out_proj.weight', 'h.2.ln_1.bias', 'h.1.attn.masked_bias', 'h.1.attn.bias', 'h.0.attn.bias', 'h.1.attn.out

{'input_ids': tensor([[ 40, 416,  79,  12, 230,  89, 231, 432, 301, 212, 933]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
torch.return_types.topk(
values=tensor([[[0.3175, 0.3173],
         [0.3451, 0.3136],
         [0.4061, 0.3571],
         [0.3755, 0.3632],
         [0.3272, 0.3082],
         [0.4069, 0.3611],
         [0.3705, 0.3377],
         [0.4229, 0.3794],
         [0.3115, 0.3054],
         [0.3394, 0.3333],
         [0.3661, 0.3298]]], grad_fn=<TopkBackward0>),
indices=tensor([[[904,  65],
         [278, 901],
         [366, 437],
         [856, 980],
         [121, 873],
         [296, 566],
         [ 39, 483],
         [415, 250],
         [314, 775],
         [393, 131],
         [639, 689]]]))


In [32]:
model.cpu()
print(inputs)
a=model.generate(**inputs,num_return_sequences=3,do_sample=True,max_new_tokens=10,eos_token_id=516,
                                    pad_token_id=516)
print(tokenizer.decode([121,111,434,122,199]))

{'input_ids': tensor([[ 40, 416,  79,  12, 230,  89, 231, 432, 301, 212, 933]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
�� un�er


In [6]:
from torch.utils.data import DataLoader
from tqdm import tqdm
def gen_topk_pos(dataset,model):
    data_loader = DataLoader(dataset)
    all_pred_ids = []
    all_pred_probs = []
    for inputs in tqdm(data_loader, desc='Sampling topk positions'):
        with torch.no_grad():
            model_inputs = {k: v.to('cuda:0')[:,:512] for k, v in inputs.items() if k != 'target_mask'}
            # print(model_inputs['input_ids'].shape,model_inputs['attention_mask'].shape)
            out = model(**model_inputs)
            api_prob_at_idx = torch.softmax(out.logits, dim=-1)[:, :, 114] # 采样所有生成的token中，[的概率
            api_prob_at_idx[inputs['target_mask'][:,:512] == 0] = -10000 # 将padding的位置的概率设为-10000, FIXME: 写的有问题
            api_topk = api_prob_at_idx.topk(2) # 采样TopK的位置
            api_topk_idx, api_topk_prob = api_topk.indices, api_topk.values
            all_pred_ids.append(api_topk_idx.detach())
            all_pred_probs.append(api_topk_prob.detach())
            torch.cuda.empty_cache()
    a,b= torch.cat(all_pred_ids, 0), torch.cat(all_pred_probs, 0)
    print(b)
    a,b=a.cpu().numpy(),b.cpu().numpy()
    for i in range(2):
        dataset=dataset.add_column('pos',a[:,i]).add_column('pro',b[:,i])
        
    return dataset
#1.2e-3
model.eval()
model.to('cuda:0')
dataset_a=gen_topk_pos(dataset_n,model)

Sampling topk positions: 100%|██████████| 25/25 [00:01<00:00, 23.07it/s]


tensor([[ 1.1448e-03,  1.1434e-03],
        [ 1.1199e-03,  1.1196e-03],
        [ 1.2109e-03,  1.1229e-03],
        [ 1.2744e-03,  1.2742e-03],
        [ 1.2006e-03,  1.0722e-03],
        [ 1.2725e-03,  1.2722e-03],
        [ 1.2323e-03,  1.1061e-03],
        [ 1.2750e-03,  1.2749e-03],
        [ 1.1952e-03,  1.1950e-03],
        [-1.0000e+04, -1.0000e+04],
        [ 1.2136e-03,  1.2124e-03],
        [ 1.2760e-03,  1.1462e-03],
        [ 1.1992e-03,  1.1978e-03],
        [ 1.1894e-03,  1.1548e-03],
        [ 1.2755e-03,  1.1647e-03],
        [ 1.1603e-03,  1.1595e-03],
        [ 1.2722e-03,  1.2071e-03],
        [ 1.2749e-03,  1.2022e-03],
        [ 1.2719e-03,  1.2717e-03],
        [ 1.2674e-03,  1.2667e-03],
        [ 1.2140e-03,  1.2000e-03],
        [ 1.2902e-03,  1.2897e-03],
        [ 1.1966e-03,  1.1896e-03],
        [ 1.2704e-03,  1.2691e-03],
        [ 1.2675e-03,  1.1923e-03]], device='cuda:0')


In [8]:
print(dataset_a)
print(dataset_a.__getitem__(9))

Dataset({
    features: ['input', 'label', 'prompt', 'input_ids', 'attention_mask', 'target_mask', 'pos', 'pro'],
    num_rows: 25
})
{'input_ids': tensor([ 57, 418, 194, 226,  75, 301, 227, 882, 798,  83, 227, 196, 247, 222,
         67, 306, 208, 210, 240,  48,  41, 227, 196, 224,  73, 262,  69, 218,
        601,  88,  84,  14, 158,  52, 195, 798,  83, 392, 456, 331,  76,  80,
        407, 235, 267, 269, 221, 650, 291, 237, 387, 961, 227, 981, 563, 198,
        601,  88,  84,  14, 558, 235, 684, 798, 198, 240,  48,  41, 307, 730,
        223, 239,  59,  35, 222,  67, 306, 208, 210,   8,  69,  88,  80, 566,
        233,   9,  61,   2, 756, 239,  69,  88,  80, 566, 233,   2, 301, 198,
        875, 566, 233, 227, 293, 441, 286, 204,  14, 282, 350, 390, 766, 377,
        251, 520, 216, 218, 240,  48,  41, 798,  83,  26, 159, 158,  41,  78,
         80, 286,  26, 304, 324, 526,  12, 615, 490, 493, 159,  10, 428,  16,
        931, 286, 216, 238, 837,  16, 931, 286, 216,  14, 158,  47, 286,

In [10]:
from datasets import Dataset,concatenate_datasets


c = Dataset.from_dict({col: [] for col in dataset_a.column_names})
print(c)
c=concatenate_datasets([c,dataset_a])
print(c)
c=concatenate_datasets([c,dataset_a])
print(c)

Dataset({
    features: ['input', 'label', 'prompt', 'input_ids', 'attention_mask', 'target_mask', 'pos', 'pro'],
    num_rows: 0
})
Dataset({
    features: ['input', 'label', 'prompt', 'input_ids', 'attention_mask', 'target_mask', 'pos', 'pro'],
    num_rows: 25
})
Dataset({
    features: ['input', 'label', 'prompt', 'input_ids', 'attention_mask', 'target_mask', 'pos', 'pro'],
    num_rows: 50
})


In [11]:
def aaa(ds):
    ds.add_column('aa',[])
c.add_column('aa',[])
c['aa']=1
print(c)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_4057010/3414000334.py:3 in <cell line: 3>                                         │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_4057010/3414000334.py'                      │
│                                                                                                  │
│ /home/chenlida/anaconda3/lib/python3.9/site-packages/datasets/arrow_dataset.py:528 in wrapper    │
│                                                                                                  │
│    525 │   │   │   "output_all_columns": self._output_all_columns,                               │
│    526 │   │   }                                                                                 │
│    527 │   │   # apply actual function                                                           │
│ ❱  528 │   │   out: Union["Dataset", "DatasetDict"] = func(self, *args, **kwargs)                │
│    529 │   │   datasets: List["Dataset"] = list(out.values()) if isinstance(out, dict) else [ou  │
│    530 │   │   # re-apply format to the output                                                   │
│    531 │   │   for dataset in datasets:                                                          │
│                                                                                                  │
│ /home/chenlida/anaconda3/lib/python3.9/site-packages/datasets/fingerprint.py:511 in wrapper      │
│                                                                                                  │
│   508 │   │   │                                                                                  │
│   509 │   │   │   # Call actual function                                                         │
│   510 │   │   │                                                                                  │
│ ❱ 511 │   │   │   out = func(dataset, *args, **kwargs)                                           │
│   512 │   │   │                                                                                  │
│   513 │   │   │   # Update fingerprint of in-place transforms + update in-place history of tra   │
│   514                                                                                            │
│                                                                                                  │
│ /home/chenlida/anaconda3/lib/python3.9/site-packages/datasets/arrow_dataset.py:5346 in           │
│ add_column                                                                                       │
│                                                                                                  │
│   5343 │   │   _check_column_names(self._data.column_names + column_table.column_names)          │
│   5344 │   │   dataset = self.flatten_indices() if self._indices is not None else self           │
│   5345 │   │   # Concatenate tables horizontally                                                 │
│ ❱ 5346 │   │   table = concat_tables([dataset._data, column_table], axis=1)                      │
│   5347 │   │   # Update features                                                                 │
│   5348 │   │   info = dataset.info.copy()                                                        │
│   5349 │   │   info.features.update(Features.from_arrow_schema(column_table.schema))             │
│                                                                                                  │
│ /home/chenlida/anaconda3/lib/python3.9/site-packages/datasets/table.py:1808 in concat_tables     │
│                                                                                                  │
│   1805 │   tables = list(tables)                                                                 │
│   1806 │   if len(tables) == 1:                            

In [4]:
from transformers import GPT2Tokenizer
tokenizer=GPT2Tokenizer.from_pretrained("EleutherAI/gpt-j-6B")
a=tokenizer(['CALCULATOR'])['input_ids']
print(a)
print(tokenizer.decode(a[0]))


[[34, 1847, 34, 6239, 25633]]
CALCULATOR


In [1]:
import re
text = 'CALCULATOR(723010101 / 252000)'
match = re.search(r'CALCULATOR\((.+?)\)', text)
if match:
    result = match.group(1)
    print(result) # 输出: 723010101 / 252000
else:
    print('No match found')

723010101 / 252000
